In [ ]:
import keras_models as kms
import keras_forward as kf
import np_models as npms
import np_forward as npf
import tf_graphs as tfgs
import tf_forward as tff
import tools
import PyPolyChord
import PyPolyChord.settings
import inverse_priors
import polychord_tools
import output
import input_tools
import forward_tests
import prior_tests

In [ ]:
###### load training data
data = 'simple_tanh'
data_dir = '../../data/'
data_prefix = data_dir + data
x_tr, y_tr = input_tools.get_x_y_tr_data(data_prefix)
batch_size = x_tr.shape[0]

In [ ]:
###### get weight information
a1_size = 2
layer_sizes = [a1_size]
m_trainable_arr = [True, False]
b_trainable_arr = [False, False]
num_inputs = tools.get_num_inputs(x_tr)
num_outputs = tools.get_num_outputs(y_tr)
num_weights = tools.calc_num_weights3(num_inputs, layer_sizes, num_outputs, m_trainable_arr, b_trainable_arr)

In [ ]:
###### check shapes of training data
x_tr, y_tr = tools.reshape_x_y_twod(x_tr, y_tr)

In [ ]:
###### setup keras model
model = kms.slp_model(num_inputs, num_outputs, layer_sizes)
km = kf.keras_model(model, x_tr, y_tr, batch_size)
loss = 'mse' 
km.setup_LL(loss)

In [ ]:
#setup tf graph
tf_graph = tfgs.slp_graph
tfm = tff.tf_model(tf_graph, x_tr, y_tr, batch_size, layer_sizes, m_trainable_arr, b_trainable_arr)
fit_metric = 'chisq'
tfm.setup_LL(fit_metric)

In [ ]:
#set up np model
np_nn = npms.slp_nn
npm = npf.np_model(np_nn, x_tr, y_tr, batch_size, layer_sizes, m_trainable_arr, b_trainable_arr)
ll_type = 'gauss'
npm.setup_LL(ll_type)

In [ ]:
#time forward_propagation using keras model
forward_tests.forward_test_linear([km], print_out = False)

In [ ]:
#time forward_propagation using tensorflow model
forward_tests.forward_test_linear([tfm], print_out = False)

In [ ]:
#time forward_propagation using np model
forward_tests.forward_test_linear([npm], print_out = False)

In [ ]:
###### setup prior
prior_types = [7]
prior_hyperparams = [[-2., 2.]]
weight_shapes = get_weight_shapes3(num_inputs, layer_sizes, num_outputs, m_trainable_arr, b_trainable_arr)
dependence_lengths = get_degen_dependence_lengths(weight_shapes)
param_prior_types = [0]
prior = inverse_prior(prior_types, prior_hyperparams, dependence_lengths, param_prior_types, num_weights)

In [ ]:
###### time prior output from nn setup
prior_tests.nn_prior_test(prior, print_out = False)

In [ ]:
###### setup polychord
nDerived = 0
settings = PyPolyChord.settings.PolyChordSettings(num_weights, nDerived)
settings.file_root = data
settings.nlive = 200
#supress output for timing purposes TODO

In [ ]:
#time polychord with keras model
settings.base_dir = './keras_chains/'
PyPolyChord.run_polychord(km, num_weights, nDerived, settings, prior, polychord_tools.dumper)

In [ ]:
#time polychord with tf model
settings.base_dir = './tf_chains/'
PyPolyChord.run_polychord(tfm, num_weights, nDerived, settings, prior, polychord_tools.dumper)

In [ ]:
#time polychord with np model
settings.base_dir = './np_chains/'
PyPolyChord.run_polychord(npm, num_weights, nDerived, settings, prior, polychord_tools.dumper)

In [ ]:
import pstats, cProfile

In [ ]:
cProfile.runctx(function to be profiled, globals(), locals(), name of prof file)

s = pstats.Stats(name of prof file)
s.strip_dirs().sort_stats("time").print_stats()
